In [1]:
import torch
import os
import numpy as np
import pandas as pd

In [2]:
def get_inner_files(path, extension):
    files = {
        os.path.relpath(os.path.join(root, fname), start=path)
        for root, _, files in os.walk(path)
        for fname in files
        if os.path.splitext(fname)[1].lower() == f".{extension}"
    }
    return sorted(files)

def encode_mask_to_rle(mask):
    """
    mask: numpy array binary mask
    1 - mask
    0 - background
    Returns encoded run length
    """
    pixels = mask.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return " ".join(str(x) for x in runs)

In [3]:
CLASSES = [
    'finger-1', 'finger-2', 'finger-3', 'finger-4', 'finger-5',
    'finger-6', 'finger-7', 'finger-8', 'finger-9', 'finger-10',
    'finger-11', 'finger-12', 'finger-13', 'finger-14', 'finger-15',
    'finger-16', 'finger-17', 'finger-18', 'finger-19', 'Trapezium',
    'Trapezoid', 'Capitate', 'Hamate', 'Scaphoid', 'Lunate',
    'Triquetrum', 'Pisiform', 'Radius', 'Ulna',
]
ind2class = {i: v for i, v in enumerate(CLASSES)}

basepath = "/opt/ml/level2_cv_semanticsegmentation-cv-01"

bh_basepath = os.path.join(basepath, "bh")
dg_basepath = os.path.join(basepath, "dg")
hj_basepath = os.path.join(basepath, "hj")

filenames = get_inner_files(bh_basepath, 'npz')

In [4]:
def get_image_path(basepath, filename):
    image_path = os.path.join(basepath, filename)
    return image_path

In [5]:
rles = []
filename_and_class = []
weight = torch.tensor([2, 1.5, 0.7])
for filename in filenames:
    bh_image_path = get_image_path(bh_basepath, filename)
    dg_image_path = get_image_path(dg_basepath, filename)
    hj_image_path = get_image_path(hj_basepath, filename)

    outputs = []
    outputs.append(torch.from_numpy(np.load(bh_image_path)['array']))
    outputs.append(torch.from_numpy(np.load(dg_image_path)['array']))
    outputs.append(torch.from_numpy(np.load(hj_image_path)['array']))

    # outputs = torch.stack(outputs)
    # outputs = torch.mean(outputs, dim=0)
    output = (outputs[0]*weight[0] + outputs[1]*weight[1] + outputs[2]*weight[2]) / torch.sum(weight)
    output = (output > 0.5).detach().cpu().numpy()

    for c, segm in enumerate(output):
        rle = encode_mask_to_rle(segm)
        rles.append(rle)
        filename_and_class.append(f"{ind2class[c]}_{filename.split('.')[0]}.png")

IndexError: list index out of range

In [ ]:
output_path = "/opt/ml/level2_cv_semanticsegmentation-cv-01/output"
classes, filename = zip(*[x.split("_") for x in filename_and_class])
image_name = [os.path.basename(f) for f in filename]
df = pd.DataFrame(
    {
        "image_name": image_name,
        "class": classes,
        "rle": rles,
    }
)
df.to_csv(os.path.join(output_path, "Strength_lies_in_unity.csv"), index=False)

In [ ]:
origin_path = "/opt/ml/level2_cv_semanticsegmentation-cv-01/output/hrnet_48w_ocr_1024_augSet_full.csv"
compare_path = "/opt/ml/level2_cv_semanticsegmentation-cv-01/output/Strength_lies_in_unity.csv"

In [ ]:
def decode_rle_to_mask(rle, height, width):
    s = rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(height * width, dtype=np.uint8)
    
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    
    return img.reshape(height, width)

In [ ]:
from collections import defaultdict

origin_df = pd.read_csv(origin_path)
origin_image_info = defaultdict(dict)
origin_image_names = list(set(origin_df['image_name']))
for image_name, class_name, rle in zip(origin_df['image_name'], origin_df['class'], origin_df['rle']):
    origin_image_info[image_name][class_name] = rle

compare_df = pd.read_csv(compare_path)
compare_image_info = defaultdict(dict)
compare_image_names = list(set(origin_df['image_name']))
for image_name, class_name, rle in zip(compare_df['image_name'], compare_df['class'], compare_df['rle']):
    compare_image_info[image_name][class_name] = rle

In [ ]:
for image_name in origin_image_names:
    origin_pred, compare_pred = [], []
    for origin_rle, compare_rle  in zip(origin_image_info[image_name].values(), compare_image_info[image_name].values()):
        origin_pred.append(decode_rle_to_mask(origin_rle, height=2048, width=2048))
        compare_pred.append(decode_rle_to_mask(compare_rle, height=2048, width=2048))
    origin_pred = torch.from_numpy(np.stack(origin_pred, 0))
    compare_pred = torch.from_numpy(np.stack(compare_pred, 0))
    print(torch.sum(origin_pred != compare_pred))

tensor(7083)
tensor(10164)
tensor(7113)
tensor(7487)
tensor(7610)
tensor(9445)
tensor(7641)
tensor(6236)
tensor(8233)
tensor(8111)
tensor(9179)
tensor(9487)
tensor(12633)
tensor(7468)
tensor(11432)
tensor(10386)
tensor(8370)
tensor(14483)
tensor(9425)
tensor(9120)
tensor(9468)
tensor(9742)
tensor(11855)
tensor(10724)
tensor(7261)
tensor(9547)
tensor(9685)
tensor(9268)
tensor(9054)
tensor(9844)
tensor(7811)
tensor(9228)
tensor(8681)
tensor(12250)
tensor(7004)
tensor(7556)
tensor(8179)
tensor(8839)
tensor(9805)
tensor(9377)
tensor(9956)
tensor(8746)
tensor(11597)
tensor(9552)
tensor(6609)
tensor(13543)
tensor(8236)
tensor(8921)
tensor(8439)
tensor(10290)
tensor(12875)
tensor(9539)
tensor(8824)
tensor(9067)
tensor(9875)
tensor(8064)
tensor(8962)
tensor(9286)
tensor(9279)
tensor(7811)
tensor(13018)
tensor(6010)
tensor(10688)
tensor(10207)
tensor(8664)
tensor(8668)
tensor(11915)
tensor(10009)
tensor(10339)
tensor(7609)
tensor(6300)
tensor(13308)
tensor(7806)
tensor(9019)
tensor(9016)
tensor